# #511 Embedded Retaining Wall with Submerged Excavation
<i>Embedded retaining wall with submerged excavation - see LUSAS example "Submerged Excavation"</i>
***

In [ ]:
''' Inputs '''
soil_layers = [20,20]
wall_depth = 30
water_level = 2

excavation_depths = [water_level,9,9]
width_excavation = 15
width_retained   = 25

spacing_of_props = 5 #m - Prop stiffness will be reduced to reflect this spacing
prop_diameter    = 0.44 #m
prop_location    = 1.0 #m from top

width_wall = 1.265

mesh_size = 1.0

surcharge_offset = width_retained - 12
surcharge_length = 10 #m
surcharge_intensity = 5 #kpa

interface_friction_angles = [12, 22] # Corresponding to each soil material defined in the spreadsheet

### Data Checks

In [ ]:
assert wall_depth > sum(excavation_depths),       "The wall depth should be larger than the excavation"
assert wall_depth < sum(soil_layers),             "The wall depth should be smaller than the total depth of soil layers"
assert sum(soil_layers) > sum(excavation_depths), "The total depth of soil layers should be larger than the excavation"
assert water_level == excavation_depths[0],       "The first excavation is assumed to be down to the level of the water table"

### Calculated parameters

In [ ]:
embed_length = wall_depth - sum(excavation_depths)
soil_depth = sum(soil_layers)
width_problem = width_retained + width_wall + width_excavation

### Connect to LUSAS Modeller & create the new model

In [ ]:
import numpy as np
import pandas as pd
import math
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()

from m100_Tools_And_Helpers import Helpers
Helpers.initialise(lusas)

# # If there is not currently a model open, create one
# if lusas.existsDatabase():
#     raise Exception("A model is already open. Please close it before running this script.")

# Create a new model
lusas.newProject("Structural", "Embedded_Wall_Submerged_Excavation")
# Reference to the model database for convenience
db = lusas.database() # Reference to the model database for convenience
# 2D model with Y vertical
db.setAnalysisCategory("2D Inplane")
db.setVerticalDir("Y")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")
db.setTimescaleUnits("Seconds")

### Mesh Attributes

In [ ]:
# Two-phase plane strain quadrilateral elements are used for the soil
soil_mesh_attr         = db.createMeshSurface("Soil").setIrregular("QPN8P", mesh_size)
# Plane strain elements for the wall
wall_mesh_attr         = db.createMeshSurface("Wall").setIrregular("QPN8", mesh_size )

# Two-phase interface elements to model Mohr=Coulomb friction contact between the structure and soil
interface_mesh_attr    = db.createMeshLine("Interface").setSize("IPN6P", mesh_size )
# Quadratic beam elements for the prop, default 4 divisions
prop_mesh_attr         = db.createMeshLine("Prop").setSpacing("BMI3")

### Geometric Attributes

In [ ]:
# Prop
prop_geom_attr = Helpers.create_circular_section(db, "Prop", prop_diameter)

### Material Attributes

In [ ]:
soil_material_attrs:list[IFAttribute] = []
# Read the soil material definitions from the corresponding spreadsheet
df_soils = pd.read_excel("511 Soil_Inputs.xlsx", header=1)
# First 3 columns define properties, remaining are soil material definitions
if len(df_soils.columns) > 3:
    for i in range(3, len(df_soils.columns)):
        # Name and parameters
        soil_name = df_soils.iloc[:,i].name
        params = dict(zip(df_soils.iloc[:,2], df_soils.iloc[:,i]))
        # Create the material
        attr = db.createIsotropicMaterial(soil_name, params['E'], params['nu'], params['rho'])
        # Modified Mohr-Coulomb properties with Rankine cut-off
        attr.addPlasticModifiedMohrCoulomb("Rankine", params['frc'], params['dil'], 0, params['dmpFct'])
        attr.addModifiedMohrCoulombCohesion(0, params['CSngle'])
        attr.addModifiedMohrCoulombTensile(0, params['QmaxSngle'])
        attr.addModifiedMohrCoulombCompressive(0, params['QminSngle'])
        # Two Phase
        attr.addConstantWaterContentTwoPhase(0.0, params['Kf'], params['n'], 9.81, params['kx'], params['ky'], params['kz'], 
                                             params['rhoF'], params['Swc'], params['Sws'], params['Kred'], 0.0, 
                                             "Absolute value", "saturation", "Fully saturated")
        # Ko Initialisation
        attr.addKoElasticRow(0.0, params['Ko'])
        soil_material_attrs.append(attr)

# Check we have sufficient soil materials defined for the number of layers
assert len(soil_material_attrs) >= len(soil_layers), " The number of defined soil materials is less than the defined layers"

In [ ]:
# Elastic materials for the wall and anchors. Note anchors are reduced to account for horizontal spacing
wall_material_attr = db.createIsotropicMaterial("Concrete", 5.93e6, 0.3, 2.4)
prop_material_attr = db.createIsotropicMaterial("Prop", 200e6/spacing_of_props, 0.0, 7.8/spacing_of_props)

In [ ]:
# Create an interface material for each of the soil materials
soil_interface_materials:list[IFSoilStructureMaterialSet] = list()
for i, soil_attr in enumerate(soil_material_attrs):
    interface_material_attr = db.createSoilStructureMaterialSet(f"{soil_attr.getName()}/Concrete")
    interface_material_attr.setSoilInterfaceGraphType("displacement")
    interface_material_attr.addSoilNrmInterfaceRow(100.0, 0.1, 0.0, interface_friction_angles[i], 0.0, 1.0, -1.0, 0.0)
    interface_material_attr.setSoilInterfacePropertiesType("Never")
    interface_material_attr.addTwoPhaseInterface(1.0)
    soil_interface_materials.append(interface_material_attr)

### Support Attributes

In [ ]:
# Base is supported vertically and horizontally
fix_xy_support_attr = db.createSupportStructural("FixXY").setStructural("R", "R", "F", "F", "F", "F", "F", "F", "C", "F")
# Sides are supported horizontally
fix_x_support_attr  = db.createSupportStructural("FixX") .setStructural("R", "F", "F", "F", "F", "F", "F", "F", "C", "F")

# Pore Water
fix_pwp_support_attr  = db.createSupportStructural("FixPwp") .setStructural("F", "F", "F", "F", "F", "F", "F", "F", "OF", "F")
fix_rot_support_attr  = db.createSupportStructural("FixRot") .setStructural("F", "F", "F", "F", "F", "R", "F", "F", "C", "F")


### Loading Attributes

In [ ]:
# Surcharge loading is applied over a patch on the retained side.
# A variation is used to specify the extent of the load which is assigned to the whole line.
load_variation_attr = db.createVariationField("Surcharge Limits")
load_variation_attr.setField("1.0")
load_variation_attr.setCoordsType("variation")
load_variation_attr.setValue("minXset", True)
load_variation_attr.setValue("minX", surcharge_offset)
load_variation_attr.setValue("maxXset", True)
load_variation_attr.setValue("maxX", surcharge_offset+surcharge_length)

# Surcharge loading is a face loading with intensity defined by the variation
surchage_load_attr = db.createLoadingFace(f"Surchage load {surcharge_intensity}kPa").setFace(0.0, f"{surcharge_intensity}*{load_variation_attr.getName()}", 0.0, 0.0)

In [ ]:
# Hydrostatic load in excavation defined by variation from phreatic surface
hydro_variation_attr = db.createVariationField("Excavation water depth")
hydro_variation_attr.setField(f"{soil_depth-water_level}-y")

hydrostatic_load_attrs = []
for i in range(1, len(excavation_depths)):
    hydrostatic_load_attrs.append(db.createLoadingFace(f"Hydrostatic load exc {i}").setFace(0.0, f"9.81*{hydro_variation_attr.getName()}", 0.0, 0.0))

### Deactivation Attributes

In [ ]:
# Deactivate the wall and prop initially
deactivate_wall_attr = db.createDeactivate(f"Wall and prop").setDeactivate("custom", 100.0, 1.0E-6)
deactivate_wall_attr.setCustomType("activeMesh").setForceDistribution("number of increments", 1)
# Deactivate the column of soil being replaced by the wall
deactivate_soil_col_attr = db.createDeactivate(f"Soil column").setDeactivate("custom", 100.0, 1.0E-6)
deactivate_soil_col_attr.setDeactivate("activeMesh", 100.0, 1.0E-6)

# Deactivate each excavation
excavation_deactivate_attrs:list[IFAttribute] = []
for i in range(len(excavation_depths)):
    attr = db.createDeactivate(f"Excavation {i+1}").setDeactivate("custom", 100.0, 1.0E-6)
    attr.setCustomType("activeMesh").setForceDistribution("distribute over stage")
    excavation_deactivate_attrs.append(attr)

### Activation Attributes

In [ ]:
wall_activate_attr = db.createActivate("wall")
prop_activate_attr = db.createActivate("prop")

***
### Create model geometry

In [ ]:
# Determine vertical points
# Top of wall, prop location, bottom of wall, bottom of problem
y_coords = [soil_depth - d for d in [0, prop_location, wall_depth, soil_depth]]
# Soil layers
y_coords.extend( [soil_depth - d for d in np.add.accumulate(soil_layers)] )
# Excavation depths
y_coords.extend( [soil_depth - d for d in np.add.accumulate(excavation_depths)] )
# Remove duplicates and sort
y_coords = list(set(y_coords))
y_coords.sort(reverse=True)

# Coordinates of the wall
y_coords_wall = [y for y in y_coords if y >= (soil_depth - wall_depth)]

### Calculate some useful indicies to locate features

In [ ]:
# Index of the prop location
i_prop = y_coords.index(soil_depth - prop_location)
# Index of phreatic surface
i_phreatic = y_coords.index(soil_depth - water_level)
# Index of the points at the bottom of the wall
i_bottom_wall = len(y_coords_wall) - 1
# Indicies of points beneath the wall
indicies_sub_wall = [i for i in range(i_bottom_wall + 1, len(y_coords))]
# Indicies of soil layers
indicies_soil_layers = [0]+[y_coords.index(soil_depth - y) for y in  np.add.accumulate(soil_layers)]
# Indicies of horizontal lines on the retained side
indicies_horiz_retained = indicies_soil_layers[0:-1] + [i_bottom_wall]
indicies_horiz_retained.sort()

In [ ]:
# The wall will  replace the soil column so we have coincident surfaces which will be activated and deactivated
# Make the geometry unmergable so two surfaces can exist at the same location
db.options().setBoolean("newFeaturesMergeable", False)

In [ ]:
# Points
points_lhs      = [Helpers.create_point(0, y, 0) for y in y_coords]
points_soil_ret = [Helpers.create_point(width_retained, y, 0) for y in y_coords_wall]
points_wall_ret = [Helpers.create_point(width_retained, y, 0) for y in y_coords_wall]
points_wall_exc = [Helpers.create_point(width_retained + width_wall, y, 0) for y in y_coords_wall]
points_soil_exc = [Helpers.create_point(width_retained + width_wall, y, 0) for y in y_coords_wall]
points_rhs      = [Helpers.create_point(width_problem, y, 0) for y in y_coords]

In [ ]:
# Lines vertical
lines_lhs      = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_lhs, points_lhs[1:]) ]
lines_soil_ret = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_soil_ret, points_soil_ret[1:]) ]
lines_wall_ret = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_wall_ret, points_wall_ret[1:]) ]
lines_wall_exc = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_wall_exc, points_wall_exc[1:]) ]
lines_soil_exc = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_soil_exc, points_soil_exc[1:]) ]
lines_rhs      = [Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_rhs, points_rhs[1:]) ]

In [ ]:
# Lines horizontal
lines_horiz_retained  = [ Helpers.create_line_from_points(points_lhs[i],      points_soil_ret[i]) for i in indicies_horiz_retained ]
lines_horiz_soil      = [ Helpers.create_line_from_points(points_soil_ret[i], points_soil_exc[i]) for i in indicies_horiz_retained ]
lines_horiz_wall      = [ Helpers.create_line_from_points(points_wall_ret[i], points_wall_exc[i]) for i in [0, i_bottom_wall] ]
lines_horiz_excavated = [ Helpers.create_line_from_points(p1, p2) for p1, p2 in zip(points_soil_exc, points_rhs) ]
lines_horiz_sub_wall  = [ Helpers.create_line_from_points(points_lhs[i], points_rhs[i]) for i in indicies_sub_wall ]
line_prop             = Helpers.create_line_from_points(points_wall_exc[i_prop], points_rhs[i_prop])

In [ ]:
# Surface for each soil layer on the retained side and soil column
surfaces_ret  : list[IFSurface] = []
surfaces_soil : list[IFSurface] = []
for i in range(len(indicies_horiz_retained)-1):
    i1 = indicies_horiz_retained[i]
    i2 = indicies_horiz_retained[i+1]
    surfaces_ret.append( Helpers.create_surface_from_lines( [lines_horiz_retained[i]] + lines_soil_ret[i1:i2] + [lines_horiz_retained[i+1]] + lines_lhs[i1:i2] ) )
    surfaces_soil.append( Helpers.create_surface_from_lines( [lines_horiz_soil[i]] + lines_soil_exc[i1:i2] + [lines_horiz_soil[i+1]] + lines_soil_ret[i1:i2] ) )

# One surface for the whole wall
surface_wall     = Helpers.create_surface_from_lines(lines_wall_exc + [lines_horiz_wall[-1]] + lines_wall_ret + [lines_horiz_wall[0]])

# Surfaces for the excavated layers
surfaces_exc     = [Helpers.create_surface_from_lines( [ lines_rhs[i], lines_horiz_excavated[i+1], lines_soil_exc[i], lines_horiz_excavated[i] ] ) for i in range(0, i_bottom_wall) ]

# Surfaces beneath the wall
surf_sub_wall = []
# First case consider the lines at the base of the wall
horiz = [ lines_horiz_retained[-1], lines_horiz_soil[-1], lines_horiz_excavated[-1] ]
surf_sub_wall.append(Helpers.create_surface_from_lines(horiz + [ lines_rhs[i_bottom_wall], lines_horiz_sub_wall[0], lines_lhs[i_bottom_wall] ] ))
# Any additional layers
for i in indicies_sub_wall[1:-1]:
    surf_sub_wall.append(Helpers.create_surface_from_lines(list(lines_rhs[i], lines_horiz_sub_wall[i+1], lines_wall_exc[i] + lines_horiz_sub_wall[i])))


### Attribute Assignments

In [ ]:
# Soil mesh
soil_mesh_attr.assignTo("Surfaces")
# Wall mesh
wall_mesh_attr.assignTo(surface_wall)
# Prop mesh
prop_mesh_attr.assignTo(line_prop)
# Interface mesh
for i in range(i_bottom_wall):
    objs = lusas.newObjectSet().add(lines_soil_ret[i]).add(lines_wall_ret[i])
    interface_mesh_attr.assignTo(objs)
    objs = lusas.newObjectSet().add(lines_soil_exc[i]).add(lines_wall_exc[i])
    interface_mesh_attr.assignTo(objs)

In [ ]:
# Helper method to locate the soil layer index from a given depth
def get_soil_index(y:float)->int:
    depth = soil_depth
    for i, d in enumerate(soil_layers):
        if(y <= depth and y > depth - d): return i
        depth-=d
    assert(False)

In [ ]:
# Wall material
wall_material_attr.assignTo(surface_wall)
# Prop material
prop_material_attr.assignTo(line_prop)

# Soil materials assigned to each surface based on the surface depth
for surface in surfaces_ret + surfaces_soil + surfaces_exc + surf_sub_wall:
    y = surface.getCentroid()[1]
    i = get_soil_index(y)
    soil_material_attrs[i].assignTo(surface)

# Soil interface materials
for il in range(i_bottom_wall):
    y = lines_soil_exc[il].getInterpolatedPosition(0.5)[1]
    i = get_soil_index(y)
    soil_interface_materials[i].assignTo(lines_soil_ret[il])
    soil_interface_materials[i].assignTo(lines_soil_exc[il])

In [ ]:
# Geometric assignments
prop_geom_attr.assignTo(line_prop)

In [ ]:
# Support assignments
fix_x_support_attr.assignTo(lines_lhs)
fix_x_support_attr.assignTo(lines_rhs)
fix_xy_support_attr.assignTo(lines_horiz_sub_wall[-1])

# Water pressures
fix_pwp_support_attr.assignTo(points_lhs[i_phreatic])
fix_rot_support_attr.assignTo(points_rhs[i_prop])

In [ ]:
# Generate the mesh
db.resetMesh()
db.updateMesh()

***
### Analysis Stages
Apply deactivation attributes to the wall initially

In [ ]:
deactivate_wall_attr.assignTo(surface_wall)
deactivate_wall_attr.assignTo(line_prop)
deactivate_wall_attr.assignTo(lines_soil_exc)
deactivate_wall_attr.assignTo(lines_soil_ret)

In [ ]:
# Initial stage with gravity
initial_loadcase = Helpers.get_loadcase(1)
initial_loadcase.setName("Initial conditions")
initial_loadcase.addGravity(True)
# The initial loadcase must specify the nonlinear controls. Here we have a basic nonlinear analysis with a manual load increment, i.e 1 increment
initial_loadcase.setTransientControl(0)
initial_loadcase.getTransientControl().setNonlinearManual().setOutput().setConstants()
initial_loadcase.getTransientControl().setValue("dlnorm", 1.0).setValue("dtnrml", 1.0) # Displacment norms

# 2nd Stage. Activate the wall elements to represent the installation, deactivate the corresponding soil column
install_wall_loadcase = db.createLoadcase("Install wall")
install_wall_loadcase.addGravity(True)
assign = lusas.assignment().setAllDefaults().setLoadset(install_wall_loadcase)
deactivate_soil_col_attr.assignTo(surfaces_soil, assign)
wall_activate_attr.assignTo(surface_wall, assign)
wall_activate_attr.assignTo(lines_soil_exc, assign)
wall_activate_attr.assignTo(lines_soil_ret, assign)

# Consider any surcharge loading present before excavation starts
surcharge_loadcase = db.createLoadcase("Surcharge")
surcharge_loadcase.addGravity(True)

In [ ]:
# Nonlinear analyses in LUSAS have either manual or automatic incrementation.
# Generally, automatic increments carry forward loading from previous stages, whereas manual do not.
# We'll keep a list of manual loadcases to which loads are assigned as a range once all loadcases are created
manual_loadcases = []
# We also need to apply the hydrostatic load to the excavation stages
excavation_loadcases = []

In [ ]:
y = soil_depth
prev_i = 0
for i, depth in enumerate(excavation_depths):
    y-=depth
    iy = y_coords.index(y)
    # print(i, depth, y, iy)
    # Loadcase/stage for the excavation
    lc = db.createLoadcase(f"Excavation {i+1}")
    # Assignment to this loadcase
    assign = lusas.assignment().setAllDefaults().setLoadset(lc)    
    # Deactivate the excavated soil elements in this stage
    excavation_deactivate_attrs[i].assignTo(surfaces_exc[prev_i:iy], assign)
    # Deactivate the excavated interface elements in this stage
    excavation_deactivate_attrs[i].assignTo(lines_soil_exc[prev_i:iy], assign)

    # The deactivation will be applied in multiple increments, with a max of 20 increments 
    # starting at 0.1 to load factor of 1
    lc.setTransientControl(20)
    lc.getTransientControl().setNonlinearAutomatic(0.1) # Output and constants will follow initial case
    lc.getTransientControl().setValue("MaxChangeInLoadFactor", 1.0).setValue("MaxLoadFactor", 1.0)
    if i > 0:
        cntrl = lc.getTransientControl().setConstants()
        cntrl.setValue("iterStrategyType", "Minimise including residuals")
        cntrl.setValue("dlnorm", 1.0).setValue("dtnrml", 1.0)
    
    if i > 0:
        # Assign hydrostatic load to replace the excavated material
        excavation_loadcases.append((lc, lines_wall_exc[prev_i:iy], lines_horiz_excavated[iy], surfaces_exc[iy], list()))

    # Activate the prop when exposed
    if(iy >= i_prop and prev_i < i_prop):
        # Loadcase/stage for the excavation
        lc = db.createLoadcase(f"Install prop")
        lc.addGravity(True)
        # Assignment to this loadcase
        assign = lusas.assignment().setAllDefaults().setLoadset(lc)    
        prop_activate_attr.assignTo(line_prop, assign)
        manual_loadcases.append(lc)

    # Lock-in 
    if i > 0 and i < len(excavation_depths)-1:
        # Loadcase/stage for the excavation lock-in
        lc = db.createLoadcase(f"Excavation {i+1} lock-in")
        lc.addGravity(True)
        manual_loadcases.append(lc)
        # Assign hydrostatic load below the phreactic surface
        excavation_loadcases[-1][4].append(lc)

    
    prev_i = iy


In [ ]:
# Assign the loading to the manual load steps

# Surcharge load.
assign = lusas.assignment().setAllDefaults().setLoadsetSpecified(surcharge_loadcase)
for lc in manual_loadcases:
    assign.addLoadsetSpecified(lc)
surchage_load_attr.assignTo(lines_horiz_retained[0], assign)

# Hydrostatic
i = 0
for lc, lines, line_h, surface, others in excavation_loadcases:
    assign = lusas.assignment().setAllDefaults().setLoadsetSpecified(lc)
    for other in others:
        assign.addLoadsetSpecified(other)
    # identify the surface for which the face load will be assigned
    assign.addHOF(surface)
    hydrostatic_load_attrs[i].assignTo(line_h, assign)
    hydrostatic_load_attrs[i].assignTo(lines, assign)
    i+=1


In [ ]:
lusas.view().mesh().showSolid(False)
lusas.view().geometry().autoColourByAttributes("Material", True)
lusas.view().insertAnnotationLayer()